In [6]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets, Dropdown, VBox, HBox, FloatText, interactive
import scipy.signal
import time
import scipy.special

# Function to generate the Root Raised Cosine (RRC) filter
def rootRaisedCosine(nsamp, roll_off, delay):
    # Fundamental frequency and bandwidth settings
    F0 = 0.5 / nsamp  # Base frequency relative to nsamp
    Br = 1  # Bandwidth rate
    Fs = Br * nsamp  # Sampling frequency
    Td = 1 / Br  # Symbol time
    Ts = 1 / Fs  # Sampling period

    # Frequency boundaries based on roll-off factor
    F1 = F0 * (1 - roll_off)
    F2 = F0 * (1 + roll_off)

    # Calculate filter order based on nsamp and delay
    filter_order = 2 * nsamp * delay

    # Generate time vector for the filter
    t = np.arange(0, filter_order, Td)
    h = []  # Initialize filter coefficients list

    # Loop to calculate filter coefficients
    for i in range(len(t)):
        # Shift time so it is centered around zero
        t_shifted = t[i] - filter_order / 2

        # Handle the case when time is zero
        if t_shifted == 0:
            h.append(np.sqrt(2 * F0) * (1 + roll_off * ((4 / np.pi) - 1)))

        # Handle special cases related to roll-off
        elif t_shifted == 1 / 8 / roll_off / F0 or t_shifted == -1 / 8 / roll_off / F0:
            h.append((roll_off * np.sqrt(F0)) * ((1 + 2 / np.pi) * np.sin(np.pi / 4 / roll_off) + 
                                                 (1 - 2 / np.pi) * np.cos(np.pi / 4 / roll_off)))

        # General case for other time values
        else:
            # Calculate the filter coefficient based on the time-shifted value
            factor1 = np.sqrt(2 * F0) / (1 - 64 * roll_off * roll_off * F0 * F0 * t_shifted * t_shifted)
            factor2 = np.sin(2 * np.pi * F1 * t_shifted) / (2 * np.pi * F0 * t_shifted)
            factor3 = (4 * roll_off / np.pi) * np.cos(2 * np.pi * F2 * t_shifted)

            # Append the calculated filter coefficient
            h.append(factor1 * (factor2 + factor3))

    # Return the filter coefficients (h)
    return h


# Function to calculate Bit Error Rate (BER) for QAM modulation
def ber_qam(EbNo, M, roll_off, F1, F2, Br):
    # Convert frequencies from MHz to Hz and bandwidth from Mbps to Hz
    F1 = F1 * 1e6  
    F2 = F2 * 1e6  
    Br = Br * 1e6  

    # Calculate bandwidth and carrier frequency
    W = F2 - F1  # Bandwidth in Hz
    fc = F1 + W / 2  # Carrier frequency

    # Calculate the number of samples per symbol
    nsamp = int(np.ceil(2 * F2 / Br)) + 7  

    # Determine QAM modulation parameters
    L = int(np.sqrt(M))  # Size of constellation
    l = np.log2(L)  # Number of bits per symbol in one dimension
    k = 2 * l  # Total bits per QAM symbol (I and Q)
    Nsymb = 10000  # Number of symbols to simulate

    # Calculate Signal-to-Noise Ratio (SNR) in dB
    SNR = EbNo - 10 * np.log10(nsamp / k / 2)

    # Generate QAM constellation points
    core = [1 + 1j, 1 - 1j, -1 + 1j, -1 - 1j]
    mapping = core[:]
    
    # Extend the constellation for higher-order QAM
    if l > 1:
        for j in range(1, int(l)):
            mapping = list(map(lambda x: x + j * 2 * core[0], mapping))
            conj_arr = np.conj(mapping)
            mapping = mapping + conj_arr.tolist()
            conj_arr = -np.conj(mapping)
            mapping = mapping + conj_arr.tolist()

    # Generate random bit sequence
    x = np.floor(2 * np.random.rand(int(k * Nsymb), 1))
    x_temp = np.reshape(x, (int(len(x) / k), int(k)))
    
    # Map bits to QAM symbols
    xsym = []
    for i in range(len(x_temp)):
        my_str = ''
        y = x_temp[i]
        for j in range(int(np.log2(M))):
            my_str = my_str + str(int(y[j]))
        a = int(my_str, 2)
        xsym.append(a)

    # Map symbols to constellation points
    y = []
    for n in range(len(xsym)):
        y.append(mapping[xsym[n]])

    # Set filter parameters
    delay = 10
    filtorder = delay * nsamp * 2

    # Apply root raised cosine filter to the signal
    shaping_filter = rootRaisedCosine(nsamp, roll_off, delay)
    ytx = scipy.signal.upfirdn([1], y, nsamp)  # Upsample signal
    ytx = np.convolve(ytx, shaping_filter)

    # Modulate the signal with the carrier frequency
    m = np.arange(1, len(ytx) + 1)
    s = np.real(np.multiply(ytx, np.exp(1j * 2 * np.pi * fc * m / nsamp)))

    # Calculate transmitted signal power
    s_matrix = np.matrix(s)
    s_matrix = s_matrix.getH()
    s_list = s_matrix.tolist()
    Ps = 10 * np.log10(np.matmul(s, s_list) / len(s))

    # Calculate noise power based on SNR
    Pn = Ps - SNR
    n = np.sqrt(10**(Pn / 10)) * np.random.randn(1, len(ytx))

    # Add noise to the signal
    snoisy = s + n

    # Demodulate the received signal
    yrx = 2 * np.multiply(snoisy, np.exp(-1j * 2 * np.pi * fc * m / nsamp))
    yrx = yrx[0, :]
    yrx = np.convolve(yrx, shaping_filter)
    yrx = yrx[::nsamp]
    yrx = yrx[2 * delay: len(yrx) - 2 * delay]

    # Separate real and imaginary parts (I and Q components)
    yi = np.real(yrx)
    yq = np.imag(yrx)

    # Decision making: Map received values back to the closest constellation points
    q = np.arange(-L + 1, L, 2)
    for n in range(len(yrx)):
        yi[n] = q[np.argmin(np.abs(q - yi[n]))]
        yq[n] = q[np.argmin(np.abs(q - yq[n]))]

    # Calculate Bit Error Rate (BER)
    error = 0
    for i in range(len(yrx)):
        if y[i] != yi[i] + 1j * yq[i]:
            error += 1

    # Return the BER
    return error / len(x)


# Function to plot BER curve for QAM modulation
def plot_ber_qam(M, roll_off, F1, F2, Br):
    # Check if the frequency range is valid
    if F1 >= F2:
        print("Warning: F1 should be less than F2.")
        return

    # Initialize lists to store experimental and theoretical BER values
    ber_exp = []
    ber_th = []

    # Calculate the number of levels for the QAM constellation
    L = int(np.sqrt(M))

    # Loop through Eb/N0 values (in dB)
    for i in range(1, 15):
        # Append the experimental BER calculated using the `ber_qam` function
        ber_exp.append(ber_qam(i, M, roll_off, F1, F2, Br))
        
        # Calculate and append the theoretical BER for QAM
        ber_th.append(((L - 1) / (L * np.log2(L)) * 
                       scipy.special.erfc(np.sqrt(3 * np.log2(L) / (L * L - 1) * 10**(i/10)))))

    # Plot the results
    plt.figure(figsize=(10, 8))

    # Plot theoretical BER as a line
    plt.semilogy(range(1, 15), ber_th)

    # Plot experimental BER as points
    plt.semilogy(range(1, 15), ber_exp, 'o')

    # Add labels, title, legend, and grid
    plt.legend(['Theoretical', 'Simulation'])
    plt.xlabel('Eb/N0 (dB)')  # X-axis label
    plt.ylabel('Bit Error Probability')  # Y-axis label
    plt.title(f'BER curve for {M}-QAM')  # Plot title
    plt.grid(which='both')  # Enable grid for both major and minor ticks
    plt.show()  # Display the plot


# Define QAM options
qam_options = {'4-QAM': 4, '16-QAM': 16, '64-QAM': 64}
qam_selector = widgets.Dropdown(options=qam_options, value=16, description='QAM Type:')

# Define additional input boxes for roll-off, F1, F2, and Br
roll_off_input = widgets.FloatText(value=0.25, description='Roll-off:')
F1_input = widgets.FloatText(value=6.75, description='F1: (MHz)')
F2_input = widgets.FloatText(value=9.25, description='F2: (MHz)')
Br_input = widgets.FloatText(value=10, description='Br: (Mbps)')

# Create interactive widget
interactive_plot = interactive(plot_ber_qam, M=qam_selector, roll_off=roll_off_input, F1=F1_input, F2=F2_input, Br=Br_input)

input_widgets = VBox([qam_selector, roll_off_input, F1_input, F2_input, Br_input], layout=Layout(width='auto'))
plot_output = interactive_plot.children[-1]

# Display the UI components
display(input_widgets, plot_output)


Output()